### Configure Azure ML Credentials & Compute Target

In [1]:
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
import os

#Enter details of your Azure Machine Learning workspace
load_dotenv()
subscription_id = os.environ.get('SUBSCRIPTION_ID')
resource_group = os.environ.get('RESOURCE_GROUP')
workspace = os.environ.get('WORKSPACE_NAME')
datastore = os.environ.get('DATASTORE_NAME')

#connect to the workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

In [6]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "sydney-compute-cpu"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()

In [7]:
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

path = f'azureml://datastores/{datastore}/paths/'
data_type = AssetTypes.URI_FOLDER
mode = InputOutputModes.RO_MOUNT
input_data = Input(type=data_type, path=path, mode=mode)

inputs = {
    "input_data": input_data
}

### Test Datastore

In [23]:
from azure.ai.ml import command

# define the command
command_job = command(
    code="./debug",
    command="python test_datastore.py --input_data ${{inputs.input_data}}",
    inputs=inputs,
    environment="AzureML-acpt-pytorch-2.2-cuda12.1@latest",
    compute=cpu_compute_target,
    experiment_name="train-on-cpu",
    display_name="test-datastore",
    identity=None
)

### Submit the Training Job to Azure ML Compute Resource

In [8]:
from azure.ai.ml import command

# define the command
command_job = command(
    code="./",
    command="python train.py --input_data ${{inputs.input_data}}",
    inputs=inputs,
    environment="AzureML-acpt-pytorch-2.2-cuda12.1@latest",
    compute=cpu_compute_target,
    experiment_name="train-on-cpu",
    display_name="train-image-classifier"
)

In [9]:
# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/Users/sydneylester/Projects/elephant-image-classification' 'https://sydney4772397004.blob.core.windows.net/1b0adfc3-11ad-451c-806a-68d87a4f8bb3-132ijpxg3q0bufd9pl5adetir8/elephant-image-classification' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading elephant-image-classification (627.11 MBs): 100%|██████████| 627106577/627106577 [01:03<00:00, 9830798.38it/s] 




'https://ml.azure.com/runs/serene_ear_bhmwgw3c98?wsid=/subscriptions/d69de6af-cb80-43c4-9527-d8d7eb3180f5/resourcegroups/lesterjt-rg/workspaces/Sydney&tid=9308e35c-fc90-4220-b851-4342ab03d0e9'

### Register Model & Save Locally

In [ ]:
from azure.ai.ml.entities import Model

# Get the completed job
job_details = ml_client.jobs.get(returned_job.name)

# Register the model from the job's outputs
registered_model = ml_client.models.create_or_update(
    Model(
        path=f"{job_details.outputs['artifacts']}/outputs/elephant_classifier_resnet50.pth",  # Job output path
        name="image-classifier",
        description="Trained image classifier",
    )
)

ml_client.models.download(registered_model, "elephant_classifier_resnet50.pth")
